In [12]:
!pip3 install --upgrade pyiwn

In [14]:
from pyiwn import iwn

In [11]:
iwn_obj = iwn.iwn()

TypeError: 'module' object is not callable

In [17]:
iwnobj = iwn.IndoWordNet(iwn.Language.MARATHI)

2021-08-22:08:53:47,954 INFO     [iwn.py:43] Loading marathi language synsets...
2021-08-22:08:53:49,24 INFO     [utils.py:157] NumExpr defaulting to 4 threads.


In [114]:
syn = iwnobj.synsets("घर")

In [116]:
syn

[Synset('कूळ.noun.1144'),
 Synset('घर.noun.1901'),
 Synset('घर.noun.1902'),
 Synset('बीळ.noun.5034'),
 Synset('घर.noun.4653'),
 Synset('घर.noun.8339'),
 Synset('संसार.noun.36596'),
 Synset('घर.noun.51401'),
 Synset('घर.noun.30562'),
 Synset('कुटुंब.noun.28473')]

In [69]:
all_verbs = set(iwnobj.all_words(iwn.PosTag.VERB))
all_nouns = set(iwnobj.all_words(iwn.PosTag.NOUN))

In [71]:
("कमी" in all_nouns)

True

In [66]:
list(all_nouns)[:10]

['धमकावणे',
 'ओरडणे',
 'वाचविणे',
 'स्थापणे',
 'कब्जा होणे',
 'दंडणे',
 'उकडविणे',
 'पाल्हेणे',
 'समाजाविणे',
 'सरसरणे']

In [65]:
len(all_nouns)

4017

In [251]:
# Getting Marathi freqlist
corpus_words = list([pair.split()[1] for pair in open("marathi_data/mr.tok.freqlist.txt", "r").read().split("\n") if len(pair.split())==2])
freqs = list([pair.split()[0] for pair in open("marathi_data/mr.tok.freqlist.txt", "r").read().split("\n") if len(pair.split())==2])

In [252]:
frequency = {pair.split()[1]:int(pair.split()[0]) for pair in open("marathi_data/mr.tok.freqlist.txt", "r").read().split("\n") if (len(pair.split())==2)}

In [253]:
frequency["घर"]

151

In [82]:
import random
random.randint(0,1)

0

In [198]:
len(all_words)

43872

In [199]:
len(freq_words)

1073

In [392]:
MIN_FREQ = 10
N=5

In [393]:
all_words = iwnobj.all_words()
freq_words = list(filter(lambda x: x in frequency and frequency[x]>MIN_FREQ, all_words))


def get_detractors(related_words, n=10):
    
    detractors = list()
    while len(detractors)!=n:
        detractor = random.choice(freq_words)
        if detractor not in related_words and len(detractor)>2 and len(detractor.split())==1:
            detractors.append(detractor)
    
    return detractors
    
    


def get_qa(word, n=10):
    try:
        synsets = iwnobj.synsets(word)
    except KeyError:
        return None, None
    if len(synsets) == 0:
        return None, None
#     print(synsets)
    all_lemmas = [l for syn in synsets for l in syn.lemma_names() if (l!=word and len(l.split(" "))==1 and len(l)>2 and l in frequency and frequency[l]>MIN_FREQ)]
#     print(all_lemmas)
    if not all_lemmas:
        return None, None
#     print(all_lemmas)
    x = random.randint(0,len(all_lemmas)-1)
    answer = all_lemmas[x]
#     print(answer)
    detractors = get_detractors(all_lemmas+[word], n)
    return (answer, detractors)

In [394]:
# Getting query words 
query_set = list()
nouns=0
verbs=0

for idx, word in enumerate(freqlist):
#     if len(query_set)==1000:
#         break
    
    if int(freqs[idx])<MIN_FREQ:
        break
    if len(word)<=2:
        continue
        
    if word not in all_nouns and word not in all_verbs:
        continue
    
    answer, detractors = get_qa(word, n=N)
    if answer is not None:
        query_set.append((word, answer, detractors))
    if word in all_nouns:
        nouns+=1
    else:
        if word in all_verbs:
            verbs+=1
        

In [395]:
qa_set = {word:{"answer": answer, "detractors": detractors} for word, answer, detractors in query_set}


In [396]:
len(qa_set)

1183

In [397]:
query_set[5:10]

[('करणे', 'आणणे', ['अष्टपैलू', 'मॉस्को', 'प्रतिज्ञापत्र', 'लाखो', 'बघून']),
 ('हजार', '1000', ['मार्केट', 'खून', 'बाल', 'फुटबॉल', 'दुखापत']),
 ('अधिक', 'जास्त', ['लागण', 'महिला', 'जलतरण', 'अवघड', 'विपरीत']),
 ('कारण', 'ध्येय', ['अभ्यास', 'अस्वस्थ', 'तृतीय', 'पटकन', 'सुट्टी']),
 ('निर्णय', 'परीक्षा', ['काही', 'कार्यक्षमता', 'निविदा', 'उपलब्ध', 'दादर'])]

In [398]:
# Writing dataset
import json
with open("evaluation/mr.wbst.json", "w") as f:
    json.dump(qa_set, f, ensure_ascii=False, indent=2)
    